# Finding leaflets in a vesicle with other stuff

This notebook demonstrates the performance of different algorithms for finding leaflets in a vesicle that has not finished aggregating, and discusses some pitfalls in the "orientation" method. I also have a different notebook demonstrating much better performance on a [tidier vesicle](lf_vesicle.ipynb). 

I obtained this file from the [FATSLiM tutorial here](https://pythonhosted.org/fatslim/documentation/tutorials.html).

The particle groups analysed are:

* `u`: the entire vesicle
* `half`: half the membrane, randomly dispersed
* `fifth`: 1/5th the membrane, randomly dispersed

In summary, this is pretty hard. "orientation" is actually sensitive to the `cutoff`, because it considers any lipids within `cutoff` distance of each other to potentially be in the same leaflet, provided they oriented at acute angles.

In [1]:
import MDAnalysis as mda
import numpy as np
import nglview as nv
from MDAnalysis.analysis.leaflet import LeafletFinder
from MDAnalysis.tests.datafiles import DPPC_vesicle_plus

_ColormakerRegistry()

In [2]:
u = mda.Universe(DPPC_vesicle_plus)
half = u.residues[::2].atoms
fifth = u.residues[::5].atoms

In [3]:
def show_leaflets(ag, select="name PO4 GL1 ROH", n_groups=8,
                  superset=None, **kwargs):
    """
    Get and show the leaflets. The first and second largest
    leaflets are coloured. If answers are given in `superset`,
    they are compared.
    """
    u = ag.universe
    lf = LeafletFinder(ag, select=select,
                       n_groups=n_groups, **kwargs)
    n_residues = ', '.join(list(map(str, lf.sizes)))
    print(f"Found {len(lf.leaflets)} leaflets: {n_residues} residues")
    if superset is not None:
        for i, (found, known) in enumerate(zip(lf.groups, superset), 1):
            diff = found - known
            if len(diff):
                print(f"Leaflet {i} is not within superset: "
                      f"{len(diff)} extra residues")
    u.add_TopologyAttr('tempfactors')
    lf1 = lf.groups[0].residues.atoms
    lf1.tempfactors = np.zeros(len(lf1)) - 10
    lf2 = lf.groups[1].residues.atoms
    lf2.tempfactors = np.zeros(len(lf2)) + 10
    for i, leaflet in enumerate(lf.groups[2:], 1):
        lfi = leaflet.residues.atoms
        lfi.tempfactors = np.ones(len(lfi)) * i
    view = nv.show_mdanalysis(lf.selection)
    view.clear_representations()
    view.add_representation('spacefill', color_scheme='bfactor')
    return view

def get_leaflets(ag, select="name PO4 GL1 ROH", **kwargs):
    LeafletFinder(ag, select=select, **kwargs)

## Entire membrane

### Graph

"graph" does quite well; although I only see 2 leaflets + 6 blobs, whereas it's found 9 groups.

In [4]:
show_leaflets(u, method="graph")

Found 9 leaflets: 3702, 2358, 4, 50, 14, 2, 10, 2, 2 residues


NGLWidget()

![messy vesicle: graph, full](images/lf_vesicle_messy_graph_1.png)

Images are embedded to save space, instead of nglview widgets.

Below I grab these leaflets so I can use them to ascertain the correctness of the the `half` and `fifth` atom groups, where it becomes a bit hard to see which leaflet is which.

In [5]:
lf = LeafletFinder(u, select="name PO4 GL1 ROH", method="graph")
superset = lf.groups[:2]

### Spectral clustering

Spectral clustering is having big, big issues.

In [6]:
show_leaflets(u, method="spectralclustering", cutoff=80, superset=superset)

Found 8 leaflets: 964, 956, 916, 888, 886, 750, 734, 50 residues
Leaflet 1 is not within superset: 6 extra residues
Leaflet 2 is not within superset: 956 extra residues


NGLWidget()

![messy vesicle: sc, full](images/lf_vesicle_messy_sc_1.png)

Reducing the number of groups helps a little, although it's grouped most of the extra ones into the outer leaflet....

In [7]:
show_leaflets(u, method="spectralclustering", cutoff=80, superset=superset, n_groups=3)

Found 3 leaflets: 3736, 2358, 50 residues
Leaflet 1 is not within superset: 34 extra residues


NGLWidget()

![messy vesicle: sc, full, 3 groups](images/lf_vesicle_messy_sc_2.png)


I won't time this or keep comparing how "spectralclustering" works for the other examples.

### Orientation

Orientation works with the default cutoff, although whatever the 6 blobs are is not clear.

In [8]:
show_leaflets(u, method="orientation", superset=superset)

Found 8 leaflets: 3702, 2358, 28, 16, 16, 12, 6, 6 residues


NGLWidget()

![messy vesicle: orient, full](images/lf_vesicle_messy_or_1.png)

Below I demonstrate that increasing the cutoff does not *always* increase the accuracy, though (even though I've claimed somewhere that it does increase it monotonically). That's because the algorithm considers all lipids within the cutoff to potentially belong to the same leaflet, providing that they have similar orientations. This, in addition to specifying how many groups you want, means it can do weird things like the below.

In [9]:
show_leaflets(u, method="orientation", cutoff=80, superset=superset)

Found 8 leaflets: 3794, 2322, 12, 8, 2, 2, 2, 2 residues
Leaflet 1 is not within superset: 94 extra residues
Leaflet 2 is not within superset: 14 extra residues


NGLWidget()

![messy vesicle: orient, full, large cutoff](images/lf_vesicle_messy_or_2.png)

## Half the residues

### Graph

Graph does pretty well here on the default cutoff again.

In [10]:
show_leaflets(half, method="graph", superset=superset);

Found 10 leaflets: 1868, 1162, 24, 2, 4, 2, 2, 4, 2, 2 residues


![messy vesicle: graph, half](images/lf_vesicle_messy_graph_2.png)

In [11]:
%timeit get_leaflets(half, method="graph")

148 ms ± 2.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Orientation

The default settings here don't quite work, as they have pulled in 2 extra lipids.

This is likely because we have specified `n_groups=8` and it has achieved that by associating the smallest group (2 lipids) with the closest large group. 

In [12]:
show_leaflets(half, method="orientation", superset=superset);

Found 8 leaflets: 1868, 1164, 12, 10, 6, 4, 4, 4 residues
Leaflet 2 is not within superset: 2 extra residues


Well, we only really care about the 2 actual leaflets. I can take advantage of that by only choosing to find 2 groups, and by using the `min_group` keyword to specify that each leaflet should have at least 20 lipids.

In [13]:
show_leaflets(half, method="orientation", n_groups=2, min_group=20, superset=superset)

Found 2 leaflets: 1868, 1162 residues


NGLWidget()

![messy vesicle: half, orient, 2 groups](images/lf_vesicle_messy_or_3m.png)

## A fifth of the residues

### Graph

"graph" is actually doing quite well here, I think! But it's missing some and added some extra lipids.

In [14]:
show_leaflets(fifth, method="graph", cutoff=25, superset=superset)

Found 11 leaflets: 730, 462, 18, 4, 2, 2, 2, 4, 2, 2, 2 residues
Leaflet 1 is not within superset: 2 extra residues


NGLWidget()

![messy vesicle: graph, fifth](images/lf_vesicle_messy_graph_2.png)

### Orientation

In [15]:
show_leaflets(fifth, method="orientation", n_groups=2, min_group=20, 
              cutoff=30, superset=superset)

Found 2 leaflets: 754, 464 residues


NGLWidget()

![messy vesicle: orient, fifth](images/lf_vesicle_messy_or_4.png)

Again, this is sensitive to the cutoff to avoid including those extra lipids.

In [16]:
show_leaflets(fifth, method="orientation", n_groups=2, min_group=20, 
              cutoff=80, superset=superset)

Found 2 leaflets: 762, 466 residues
Leaflet 1 is not within superset: 10 extra residues
Leaflet 2 is not within superset: 4 extra residues


NGLWidget()